In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
train = pd.read_csv("../../../Dacon/open/train.csv")
test = pd.read_csv("../../../Dacon/open/test.csv")

In [3]:
from konlpy.tag import Mecab
mecab = Mecab()

In [4]:
%%time
def pos(x):
    try:
        text = ''
        for word, pos in mecab.pos(str(x)):
            if pos in ["NNG", "NNP", "VV", "VA", "VCP", "VCN", "MM", "MAG", "XPN", "SL", "SH"]:
                if type(re.search("\W+|[0-9]", word))!=re.Match and len(word)>1: 
                    # and len(word)!=1:
                    text+=" "+word
        return text.strip()
    
    except:
        pass

train["과제명"] = train["과제명"].apply(pos)
train["요약문_한글키워드"] = train["요약문_한글키워드"].apply(pos)

test["과제명"] = test["과제명"].apply(pos)
test["요약문_한글키워드"] = test["요약문_한글키워드"].apply(pos)

CPU times: user 21.1 s, sys: 76.9 ms, total: 21.2 s
Wall time: 21.2 s


In [5]:
train["concat"] = train["과제명"]+" "+train["요약문_한글키워드"]
test["concat"] = test["과제명"]+" "+test["요약문_한글키워드"]

In [7]:
from gensim.models import Word2Vec
train_tokens = [sent.split(" ") for sent in train["concat"]]
test_tokens = [sent.split(" ") for sent in test["concat"]]

model_train = Word2Vec(sentences=train_tokens, vector_size=100, alpha=0.025, window=5, min_count=1, workers=4)
model_test = Word2Vec(sentences=test_tokens, vector_size=100, alpha=0.025, window=5, min_count=1, workers=4)

/Users/ryleyun/opt/anaconda3/envs/nlppython/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [8]:
from gensim.models import KeyedVectors
train_v2w = []
test_v2w = []

for sent in train_tokens:
    temp = []
    for word in sent:
        try:
            temp.append(model_train.wv[word].tolist())
        except:
            pass
    train_v2w.append(temp)

for sent in test_tokens:
    temp = []
    for word in sent:
        try:
            temp.append(model_test.wv[word].tolist())
        except:
            pass
    test_v2w.append(temp)
        
# train_v2w = [model_train.wv[word] for sent in train_tokens for word in sent]
# test_v2w = [model_train.wv[word] for sent in test_tokens for word in sent]

In [21]:
train_v2w = np.array(train_v2w).reshape(-1,1)
test_v2w = np.array(test_v2w).reshape(-1,1)

/var/folders/s5/j2ttz_bd7d500bsb_zg9212w0000gn/T/ipykernel_9131/1025690366.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_v2w = np.array(test_v2w).reshape(-1,1)


In [22]:
from sklearn.model_selection import train_test_split
train_data, valid_data, train_label, valid_label = train_test_split(train_v2w, train["label"], test_size=0.1, random_state=42)

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

text_clf = Pipeline([("clf", MultinomialNB())])
text_clf = text_clf.fit(train_data, train_label)

gs_clf = {"clf__alpha":[0.01, 0.008]}

gs_clf = GridSearchCV(text_clf, params_clf, n_jobs=5, verbose=2)
gs_clf = gs_clf.fit(train_data, train_label)

print("Best score: {}".format(gs_clf.best_score_))
best_parametors = gs_clf.best_estimator_.get_params()

ValueError: setting an array element with a sequence.

In [ ]:
best_parametors

In [ ]:
pred = gs_clf.best_estimator_.predict(valid_data)
# pred = text_clf.predict(valid_data)
np.mean(pred == valid_label)

import pickle
with open("naive_bayes_concat_5gram.pkl", "wb") as f:
    pickle.dump(text_clf, f)

In [ ]:
pred = gs_clf.best_estimator_.predict(test["concat"])

In [ ]:
sum(pred != 0)

In [ ]:
sample_submission=pd.read_csv('./open/sample_submission.csv')
sample_submission['label']=pred
sample_submission.to_csv('naive_bayes_keyname(dic_variouspos&over1).csv', index=False)